In [ ]:
!pip install -q simpletransformers

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip

In [ ]:
import os
import re
import pandas as pd

In [ ]:
def read_snli(data_dir, is_train):
    """Read the SNLI dataset into premises, hypotheses, and labels."""
    def extract_text(s):
        # Remove information that will not be used by us
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # Substitute two or more consecutive whitespace with space
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt'
                             if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    df = pd.DataFrame(list(zip(premises, hypotheses, labels)),
               columns =["text_a", "text_b", "labels"])
    return df

In [ ]:
df = read_snli("snli_1.0", is_train=True)
df.head()

In [ ]:
train_df = df[:5000]
eval_df = df[15000:15500]
test_df = df[15500:16000]

In [ ]:
from simpletransformers.classification import ClassificationModel


# Training arguments
train_args = {
    'evaluate_during_training': True,
    'num_train_epochs': 4,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 32,
    'overwrite_output_dir': True,
    'wandb_project': "snli_distilbert"
}

model = ClassificationModel('albert', 'albert-base-v2', num_labels=3, use_cuda=True, cuda_device=0, args=train_args)

# Train model
model.train_model(train_df, eval_df=eval_df)
result, model_outputs, wrong_predictions = model.eval_model(test_df)